# LBM 1D Code Example

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import clear_output
from IPython.display import HTML

## Problem Definiton

Define diffusivity $D$, domain length $L$ and maximum time $t_{max}$

In [2]:
D = 0.07
L = 1.0
t_max = 0.1

In [3]:
def ic(x, L):
    # return np.exp(-((x - L / 2) / 0.1) ** 2)
    return np.sin(2*np.pi*x)*0

In [4]:
def source(x, t, L, D):
    return np.sin(2*np.pi*(x-t/3))-2/3*np.pi*np.cos(2*np.pi*(x-t/3))*t + D*4*(np.pi)**2 * np.sin(2*np.pi*(x-t/3))*t

In [5]:
def u_exact(x, t, L):
    return np.sin(2*np.pi*(x-t/3))*t

## Numerical parameters
Define number of lattice nodes $nx$, number of time steps $nt$ and the weighting factor $\theta$. The refinement factor $rf<1$ allows to increase resolution according to the diffusive scaling.

In [6]:
nx = 32
nt = 100
theta = 1/3
rf = 1

## Preprocessing
 - Compute the discretization to be used in the analysis: $nx\leftarrow nx/rf$ and $nt\leftarrow nt/rf^2$
 - Determine lattice spacing $\Delta x = L/nx$ and time step size $\Delta t = t_{max}/nt$
 - Setup an array of lattice node locations (assuming $\Delta x/2$ padding at the left and right side of the $(0,L)$ domain
 - Calculate the relaxation rate $\omega$

In [7]:
nx = int(nx/rf);
nt = int(nt/rf**2);
dx = L/nx;
dt = t_max/nt;
omega = 1/(D*dt/((dx**2)*theta)+0.5);

## Equilibrium
Define the local equilibrium $f^{eq}(u)$

In [8]:
def feq(u):
    return np.array([
        (1 - theta) * u,
        (theta / 2) * u,
        (theta / 2) * u
    ])

## Definition of a single LBM time step
 - Compute zeroth-order moment $u^{num} = \sum_i f_i$
 - Perform collision
 - Perform streaming
 - Apply periodic boundary conditions

In [9]:
def timestep(f, omega, phi_dt):
    unum = np.sum(f, axis=0) + phi_dt/2
    fpost = omega*feq(unum) + (1-omega)*f
    fpost[0,:] = fpost[0,:] + (1-omega/2)*phi_dt

    f[0,:] = fpost[0,:]
    f[1,1:] = fpost[1,:-1]
    f[2,:-1] = fpost[2,1:]

    f[1,0] = fpost[1,-1]
    f[2,-1] = fpost[2,0]
    
    return f, unum

## Time step loop

Define and execute lattice Boltzmann method with given initial condition for $nt$ time steps. The lattice nodes are located at a $\Delta x/2$ offset from the domain boundary, i.e. $x_{domain} = \{1/2\Delta x, 3/2\Delta x, \dots, L-1/2\Delta x\}$

In [10]:
def lbm(L,dx,dt,nx,nt,omega,D):

    u_out = np.zeros((nt,nx))
    u_ex = np.zeros((nt,nx))
    x_domain = np.arange(dx/2, L, dx)
    
    f = feq(ic(x_domain,L)) - dt*source(x_domain, 0, L, D)/2
    for it in range(nt):
        phi = source(x_domain, it*dt, L, D)
        u_ex[it,:] = u_exact(x_domain, it*dt, L)
        f, u_out[it,:] = timestep(f, omega, phi*dt)
    return x_domain, u_out, u_ex

In [11]:
x_domain, u_out, u_ex = lbm(L,dx,dt,nx,nt,omega,D)

## Visualization

Create animation of $u^{num}$

In [12]:
def animate_u_interval(u_out, x_domain, interval=1, u_out2=None, labels=("u₁", "u₂")):
    """
    Animate one or two time-varying fields over x_domain, showing the first and last frame,
    and intermediate frames at a specified interval.

    Parameters:
        u_out (ndarray): shape (nt, nx), main data (rows = time steps)
        x_domain (ndarray): spatial coordinates
        interval (int): step between frames (skip this many frames)
        u_out2 (ndarray, optional): second dataset of same shape to plot alongside u_out
        labels (tuple): labels for the legend (only used if u_out2 is given)
    """
    nt = u_out.shape[0]

    # Select frames: always include first & last, skip in between
    if nt > 2:
        frames = [0] + list(range(interval, nt - 1, interval)) + [nt - 1]
    else:
        frames = list(range(nt))

    fig, ax = plt.subplots(figsize=(6, 4))
    line1, = ax.plot([], [], lw=2, label=labels[0])
    if u_out2 is not None:
        line2, = ax.plot([], [], lw=2, label=labels[1], linestyle='--')
    else:
        line2 = None

    ax.set_xlim(x_domain[0], x_domain[-1])
    ax.set_ylim(1.1*min(np.min(u_out), np.min(u_out2) if u_out2 is not None else np.min(u_out)),
                1.1*max(np.max(u_out), np.max(u_out2) if u_out2 is not None else np.max(u_out)))
    ax.set_xlabel("x")
    ax.set_ylabel("u")
    ax.grid(True)
    if u_out2 is not None:
        ax.legend()

    def init():
        line1.set_data([], [])
        if line2:
            line2.set_data([], [])
        return (line1, line2) if line2 else (line1,)

    def update(frame_idx):
        frame = frames[frame_idx]
        line1.set_data(x_domain, u_out[frame, :])
        if line2 is not None:
            line2.set_data(x_domain, u_out2[frame, :])
        ax.set_title(f"t = {frame}")
        return (line1, line2) if line2 else (line1,)

    ani = FuncAnimation(fig, update, frames=len(frames), init_func=init,
                        blit=True, interval=100)
    plt.close(fig)  # prevent static plot
    return HTML(ani.to_jshtml())

In [13]:
animate_u_interval(u_out, x_domain, 10, u_ex, ("num","exact"))

In [14]:
l2_exact = np.sqrt(dx) * np.linalg.norm(u_ex[-1,:])
l2 = np.sqrt(dx) * np.linalg.norm(u_out[-1,:] - u_ex[-1,:]) / l2_exact
linf = np.max(np.abs(u_out[-1,:] - u_ex[-1,:])) / l2_exact
print(f"l2 = {l2}")
print(f"linf = {linf}")

l2 = 0.009970514190773154
linf = 0.014081069094436018
